Implement a transition diagram to recognize the first occurrence of a token which can be an identifier, constant, or keyword and return as token <token type, actual token>.  

In [ ]:
#include <stdio.h>
#include <ctype.h>

int isId(char* t) {
    return (isalpha(*t) || *t == '_') && !isalnum(t[strspn(t+1, "_0-9a-zA-Z")]);
}

int isNum(char* t) {
    return isdigit(*t) && !isalpha(t[strspn(t, "0-9")]);
}

int isKey(char* t) {
    static char* kw[] = {"int", "float", "char"};
    for (int i = 0; i < 3; i++)
        if (!strcmp(t, kw[i])) return 1;
    return 0;
}

int main() {
    char str[100], *token;
    fgets(str, 100, stdin);
    str[strcspn(str, "\n")] = 0;

    for (token = strtok(str, "=+"); token; token = strtok(NULL, "=+"))
        printf("<%s, %s>\n", isId(token) ? "id" : isNum(token) ? "const" : isKey(token) ? "keyword" : "unknown", token);

    return 0;
}


Write a program that reads single productions of the form “ A → α1, A →α2, A→ α3 ... “ and make them into a single production “A → α1 / α2 /α3 ...” and display

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define MAX_RIGHT 100 // Max size for the right-hand side

int main() {
    char productions[26][MAX_RIGHT] = {0}; // Initialize all to zero
    char input[100], left, *right;
    int index;

    printf("Enter productions (e.g., A->a). Enter 'end' to finish.\n");

    while (1) {
        if (scanf("%99s", input) != 1 || strcmp(input, "end") == 0) break; // Read input

        left = input[0];
        right = strchr(input, '>') + 1;
        index = left - 'A'; // Convert char to index

        if (productions[index][0] != '\0') {
            strcat(productions[index], " / "); // Add delimiter if not the first production
        }

        strcat(productions[index], right); // Concatenate the new production
    }

    for (int i = 0; i < 26; i++) { // Print non-empty productions
        if (productions[i][0] != '\0') {
            printf("%c -> %s\n", i + 'A', productions[i]);
        }
    }

    return 0;
}



Code: 2Write a program that reads single productions of the form “ A → α1 , B →α2, B→ α3 ... “ and build CFG and display. (Identify Terminals, Non-terminals, productions, and start symbol)



In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>

#define MAX_PRODUCTIONS 100
#define MAX_SYMBOLS 50

int main() {
    char productions[MAX_PRODUCTIONS][MAX_SYMBOLS];
    char nonTerminals[MAX_SYMBOLS];
    char terminals[MAX_SYMBOLS];
    int prodCount = 0, nonTerminalCount = 0, terminalCount = 0;

    printf("Enter productions (e.g., A->aB|b). Enter 'end' to finish.\n");

    while (1) {
        char input[MAX_SYMBOLS];
        scanf("%99s", input);
        if (strcmp(input, "end") == 0) break;

        char *right = strchr(input, '>') + 1;
        char left = input[0];

        nonTerminals[nonTerminalCount++] = left;
        strcpy(productions[prodCount++], right);

        for (int i = 0; right[i] != '\0'; i++) {
            if (!isupper(right[i]) && right[i] != '|' && right[i] != '-' && right[i] != '>') {
                terminals[terminalCount++] = right[i];
            }
        }
    }

    printf("\nNon-terminals: ");
    for (int i = 0; i < nonTerminalCount; i++) {
        printf("%c ", nonTerminals[i]);
    }

    printf("\nTerminals: ");
    for (int i = 0; i < terminalCount; i++) {
        printf("%c ", terminals[i]);
    }

    printf("\nProductions:\n");
    for (int i = 0; i < prodCount; i++) {
        printf("%c -> %s\n", nonTerminals[i], productions[i]);
    }

    printf("Start Symbol: %c\n", nonTerminals[0]);

    return 0;
}


WEEK – 7Write a program that takes grammar and checks for left recursion (may contain left recursion in more than one production) and, if found, eliminate it and display the resultant grammar.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

typedef struct Production {
    char left;
    char right[100];
    struct Production *next;
} Production;

Production *createProduction(char left, char *right) {
    Production *newProd = (Production *)malloc(sizeof(Production));
    if (newProd == NULL) {
        printf("Memory allocation failed\n");
        exit(1);
    }
    newProd->left = left;
    strcpy(newProd->right, right);
    newProd->next = NULL;
    return newProd;
}

void addProduction(Production **head, char left, char *right) {
    Production *newProd = createProduction(left, right);
    if (*head == NULL) {
        *head = newProd;
    } else {
        newProd->next = *head;
        *head = newProd;
    }
}

void printProductions(Production *head) {
    while (head != NULL) {
        printf("%c -> %s\n", head->left, head->right);
        head = head->next;
    }
}

void eliminateLeftRecursion(Production **head) {
    Production *current = *head;
    while (current != NULL) {
        char *right = current->right;
        if (right[0] == current->left) {
            char alpha[100] = "", beta[100] = "", newRight[100] = "";
            char *token = strtok(right, "|");
            while (token != NULL) {
                if (token[0] == current->left) {
                    snprintf(alpha, sizeof(alpha), "%s%s|", alpha, token + 1);
                } else {
                    snprintf(beta, sizeof(beta), "%s%s%c'|", beta, token, current->left);
                }
                token = strtok(NULL, "|");
            }
            alpha[strlen(alpha) - 1] = '\0';
            beta[strlen(beta) - 1] = '\0';

            snprintf(newRight, sizeof(newRight), "%s", beta);
            strcpy(current->right, newRight);

            char newLeft = current->left + 1;
            addProduction(head, newLeft, alpha);
        }
        current = current->next;
    }
}

int main() {
    Production *head = NULL;
    printf("Enter productions (e.g., A->aB | b). Enter 'end' to finish.\n");
    char input[200];

    while (1) {
        fgets(input, sizeof(input), stdin);
        if (input[strlen(input) - 1] == '\n') {
            input[strlen(input) - 1] = '\0';
        }
        if (strcmp(input, "end") == 0) break;

        char left = input[0];
        char *right = strchr(input, '>') + 1;

        addProduction(&head, left, right);
    }

    printf("\nOriginal Productions:\n");
    printProductions(head);

    eliminateLeftRecursion(&head);

    printf("\nProductions after eliminating left recursion:\n");
    printProductions(head);

    return 0;
}
